In [3]:
import pandas as pd
import yfinance as yf
import requests
from datetime import datetime, timedelta


Индекс Moex

In [4]:
import requests
import pandas as pd

url = "https://iss.moex.com/iss/history/engines/stock/markets/index/boards/SNDX/securities/IMOEX.json"

params = {
    'from': '2017-01-01',
    'till': '2021-12-31',
    'start': 0
}

all_records = []

while True:
    response = requests.get(url, params=params)
    data = response.json()

    columns = data['history']['columns']
    rows = data['history']['data']

    if not rows:
        break

    df = pd.DataFrame(rows, columns=columns)
    all_records.append(df)

    # Следующая страница
    params['start'] += 100

# Объединяем все страницы
imoex_df = pd.concat(all_records, ignore_index=True)

# Оставляем нужные столбцы
imoex_df = imoex_df[['TRADEDATE', 'CLOSE']]

# Переименовываем столбец
imoex_df = imoex_df.rename(columns={'TRADEDATE': 'date'})

# Сортируем по дате
imoex_df['date'] = pd.to_datetime(imoex_df['date'])
imoex_df = imoex_df.sort_values('date')

# Создаём список всех дат с 01.01.2017 по 31.12.2021
all_dates = pd.date_range(start='2017-01-01', end='2021-12-31', freq='D')

# Индексируем DataFrame по дате
imoex_df.set_index('date', inplace=True)

# Пересоздаём DataFrame с полным диапазоном дат и заполняем пропуски значением 0
full_df = pd.DataFrame(index=all_dates)
full_df = full_df.join(imoex_df, how='left')
full_df['CLOSE'].fillna(0, inplace=True)

# Сбрасываем индекс, чтобы дата была столбцом
full_df.reset_index(inplace=True)
full_df.rename(columns={'index': 'date'}, inplace=True)
full_df['date'] = full_df['date'].astype('object')
full_df['date'] = pd.to_datetime(full_df['date']).dt.strftime('%Y-%m-%d')
# Сохраним в CSV
imoex_df.to_csv('imoex_2017_2021.csv', index=False, encoding='utf-8')

print(full_df)


            date    CLOSE
0     2017-01-01     0.00
1     2017-01-02     0.00
2     2017-01-03  2285.43
3     2017-01-04  2263.90
4     2017-01-05  2220.35
...          ...      ...
1821  2021-12-27  3740.63
1822  2021-12-28  3777.56
1823  2021-12-29  3741.07
1824  2021-12-30  3787.26
1825  2021-12-31     0.00

[1826 rows x 2 columns]


/var/folders/6c/xwwfz00d3rsbq6g8ymk8cmjc0000gn/T/ipykernel_81428/3585956794.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  imoex_df = pd.concat(all_records, ignore_index=True)
/var/folders/6c/xwwfz00d3rsbq6g8ymk8cmjc0000gn/T/ipykernel_81428/3585956794.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace

Курс доллара

In [5]:
data = yf.download("USDRUB=X", start="2017-01-01", end="2021-12-31")
df_data = data[['Close']].reset_index()
df_data.columns = ['date', 'USD_Rate']
df_data['date'] = pd.to_datetime(df_data['date']).dt.strftime('%Y-%m-%d')

print(df_data.head())

df_data.to_csv("usd_rub_2017_2021.csv", index=False)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

         date   USD_Rate
0  2017-01-02  61.231300
1  2017-01-03  61.228298
2  2017-01-04  60.923000
3  2017-01-05  60.365002
4  2017-01-06  59.401199


Инфляция

In [6]:
def get_inflation_rate(year):
    url = f"http://api.worldbank.org/v2/country/RU/indicator/FP.CPI.TOTL.ZG?date={year}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for entry in data[1]:
            if entry['date'] == str(year):
                return entry['value']
    return None

def generate_daily_inflation_rate(start_year, end_year):
    all_data = []


    for year in range(start_year, end_year + 1):
        inflation_rate = get_inflation_rate(year)
        if inflation_rate is not None:

            is_leap_year = (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0))
            days_in_year = 366 if is_leap_year else 365


            start_date = datetime(year, 1, 1)
            for day in range(days_in_year):
                date = start_date + timedelta(days=day)
                all_data.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'inflation_rate': inflation_rate
                })


    df = pd.DataFrame(all_data)
    return df


df_daily_inflation_rate = generate_daily_inflation_rate(2017, 2021)


df_daily_inflation_rate.to_csv('inflation_rate_daily_2017_2021.csv', index=False)


print("Результаты сохранены в файл 'inflation_rate_daily_2017_2021.csv'")


Результаты сохранены в файл 'inflation_rate_daily_2017_2021.csv'


Торговый баланс ( ТБ = экспорт - импорт)

In [7]:

def get_trade_balance(year):

    export_url = f"http://api.worldbank.org/v2/country/RU/indicator/NE.EXP.GNFS.CD?date={year}&format=json"
    export_response = requests.get(export_url)
    if export_response.status_code == 200:
        export_data = export_response.json()
        for entry in export_data[1]:
            if entry['date'] == str(year):
                export_value = entry['value']
    else:
        return None


    import_url = f"http://api.worldbank.org/v2/country/RU/indicator/NE.IMP.GNFS.CD?date={year}&format=json"
    import_response = requests.get(import_url)
    if import_response.status_code == 200:
        import_data = import_response.json()
        for entry in import_data[1]:
            if entry['date'] == str(year):
                import_value = entry['value']
    else:
        return None


    if export_value is not None and import_value is not None:
        trade_balance = export_value - import_value
        return trade_balance
    return None


def generate_daily_trade_balance(start_year, end_year):
    all_data = []


    for year in range(start_year, end_year + 1):
        trade_balance = get_trade_balance(year)
        if trade_balance is not None:

            is_leap_year = (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0))
            days_in_year = 366 if is_leap_year else 365


            start_date = datetime(year, 1, 1)
            for day in range(days_in_year):
                date = start_date + timedelta(days=day)
                all_data.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'trade_balance': trade_balance
                })


    df = pd.DataFrame(all_data)
    return df


df_daily_trade_balance = generate_daily_trade_balance(2017, 2021)


df_daily_trade_balance.to_csv('trade_balance_daily_2017_2021.csv', index=False)


print("Результаты сохранены в файл 'trade_balance_daily_2017_2021.csv'")


Результаты сохранены в файл 'trade_balance_daily_2017_2021.csv'


уровень безработицы

In [8]:
def get_unemployment_rate(year):
    url = f"http://api.worldbank.org/v2/country/RU/indicator/SL.UEM.TOTL.ZS?date={year}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for entry in data[1]:
            if entry['date'] == str(year):
                return entry['value']
    return None


def generate_daily_unemployment_rate(start_year, end_year):
    all_data = []


    for year in range(start_year, end_year + 1):
        unemployment_rate = get_unemployment_rate(year)
        if unemployment_rate is not None:

            is_leap_year = (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0))
            days_in_year = 366 if is_leap_year else 365


            start_date = datetime(year, 1, 1)
            for day in range(days_in_year):
                date = start_date + timedelta(days=day)
                all_data.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'unemployment_rate': unemployment_rate
                })


    df = pd.DataFrame(all_data)
    return df


df_daily_unemployment_rate = generate_daily_unemployment_rate(2017, 2021)
df_daily_unemployment_rate.to_csv('unemployment_rate_daily_2017_2021.csv', index=False)

print("Результаты сохранены в файл 'unemployment_rate_daily_2017_2021.csv'")


Результаты сохранены в файл 'unemployment_rate_daily_2017_2021.csv'


ввп на душу населения

In [9]:
def get_gdp_per_capita(year):
    url = f"http://api.worldbank.org/v2/country/RU/indicator/NY.GDP.PCAP.CD?date={year}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for entry in data[1]:
            if entry['date'] == str(year):
                return entry['value']
    return None

def generate_daily_gdp_per_capita(start_year, end_year):
    all_data = []


    for year in range(start_year, end_year + 1):
        gdp_per_capita = get_gdp_per_capita(year)
        if gdp_per_capita is not None:

            is_leap_year = (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0))
            days_in_year = 366 if is_leap_year else 365


            start_date = datetime(year, 1, 1)
            for day in range(days_in_year):
                date = start_date + timedelta(days=day)
                all_data.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'gdp_per_capita': gdp_per_capita
                })


    df = pd.DataFrame(all_data)
    return df

df_daily_gdp_per_capita = generate_daily_gdp_per_capita(2017, 2021)
df_daily_gdp_per_capita.to_csv('gdp_per_capita_daily_2017_2021.csv', index=False)

print("Результаты сохранены в файл 'gdp_per_capita_daily_2017_2021.csv'")


Результаты сохранены в файл 'gdp_per_capita_daily_2017_2021.csv'


индекс потребительского доверия

In [10]:
url = "http://api.worldbank.org/v2/country/RU/indicator/NE.CON.TOTL.ZS?date=2017:2021&format=json"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    if len(data) > 1 and len(data[1]) > 0:

        result = []


        for record in data[1]:
            year = str(record['date'])
            value = record['value']


            start_date = datetime.strptime(year, "%Y")
            days_in_year = 366 if (start_date.year % 4 == 0 and (start_date.year % 100 != 0 or start_date.year % 400 == 0)) else 365

            for day in range(days_in_year):
                date = start_date + timedelta(days=day)
                result.append([date.strftime("%Y-%m-%d"), value])


        consumer_expenses_df = pd.DataFrame(result, columns=['date', 'consumer_confidence_index'])
        consumer_expenses_df.to_csv('consumer_expenses_daily_2017_2021.csv', index=False, encoding='utf-8')

        print("Данные успешно записаны в файл 'consumer_expenses_daily_2017_2021.csv'.")
    else:
        print("Данные для указанного индикатора и периода не найдены.")
else:
    print(f"Ошибка при получении данных: {response.status_code}")


Данные успешно записаны в файл 'consumer_expenses_daily_2017_2021.csv'.


# **Социальные и демографические**

плотность населения

In [11]:
def get_population_density(year):
    url = f"http://api.worldbank.org/v2/country/RU/indicator/EN.POP.DNST?date={year}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for entry in data[1]:
            if entry['date'] == str(year):
                return entry['value']
    return None

def generate_daily_population_density(start_year, end_year):
    all_data = []


    for year in range(start_year, end_year + 1):
        population_density = get_population_density(year)
        if population_density is not None:

            is_leap_year = (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0))
            days_in_year = 366 if is_leap_year else 365

            start_date = datetime(year, 1, 1)
            for day in range(days_in_year):
                date = start_date + timedelta(days=day)
                all_data.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'population_density': population_density
                })


    df = pd.DataFrame(all_data)
    return df


df_daily_population_density = generate_daily_population_density(2017, 2021)
df_daily_population_density.to_csv('population_density_daily_2017_2021.csv', index=False)

print("Результаты сохранены в файл 'population_density_daily_2017_2021.csv'")


Результаты сохранены в файл 'population_density_daily_2017_2021.csv'


Возрастная структура населения

In [12]:
def get_age_structure(year):
    indicators = {
        '0-14': 'SP.POP.0014.TO.ZS',  # Доля населения 0-14 лет
        '15-64': 'SP.POP.1564.TO.ZS',  # Доля населения 15-64 лет
        '65+': 'SP.POP.65UP.TO.ZS'     # Доля населения 65+
    }
    data = {}
    for age_group, indicator in indicators.items():
        url = f"http://api.worldbank.org/v2/country/RU/indicator/{indicator}?date={year}&format=json"
        response = requests.get(url)
        if response.status_code == 200:
            result = response.json()
            for entry in result[1]:
                if entry['date'] == str(year):
                    data[age_group] = entry['value']
        else:
            data[age_group] = None
    return data


def generate_daily_age_structure(start_year, end_year):
    all_data = []


    for year in range(start_year, end_year + 1):
        age_structure = get_age_structure(year)
        if age_structure:

            is_leap_year = (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0))
            days_in_year = 366 if is_leap_year else 365


            start_date = datetime(year, 1, 1)
            for day in range(days_in_year):
                date = start_date + timedelta(days=day)
                all_data.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'age_0_14': age_structure.get('0-14'),
                    'age_15_64': age_structure.get('15-64'),
                    'age_65_plus': age_structure.get('65+')
                })


    df = pd.DataFrame(all_data)
    return df


df_daily_age_structure = generate_daily_age_structure(2017, 2021)
df_daily_age_structure.to_csv('age_structure_daily_2017_2021.csv', index=False)

print("Результаты сохранены в файл 'age_structure_daily_2017_2021.csv'")


Результаты сохранены в файл 'age_structure_daily_2017_2021.csv'


миграционный баланс

In [13]:
def get_migration_balance(year):
    url = f"http://api.worldbank.org/v2/country/RU/indicator/SM.POP.NETM?date={year}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for entry in data[1]:
            if entry['date'] == str(year):
                return entry['value']
    return None


def generate_daily_migration_balance(start_year, end_year):
    all_data = []


    for year in range(start_year, end_year + 1):
        migration_balance = get_migration_balance(year)
        if migration_balance is not None:

            is_leap_year = (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0))
            days_in_year = 366 if is_leap_year else 365


            start_date = datetime(year, 1, 1)
            for day in range(days_in_year):
                date = start_date + timedelta(days=day)
                all_data.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'migration_balance': migration_balance
                })


    df = pd.DataFrame(all_data)
    return df


df_daily_migration_balance = generate_daily_migration_balance(2017, 2021)
df_daily_migration_balance.to_csv('migration_balance_daily_2017_2021.csv', index=False)

print("Результаты сохранены в файл 'migration_balance_daily_2017_2021.csv'")


Результаты сохранены в файл 'migration_balance_daily_2017_2021.csv'


Уровень цифровизации

In [14]:
def get_digitalization_level(year):
    url = f"http://api.worldbank.org/v2/country/RU/indicator/IT.NET.USER.ZS?date={year}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for entry in data[1]:
            if entry['date'] == str(year):
                return entry['value']
    return None


def generate_daily_digitalization_level(start_year, end_year):
    all_data = []


    for year in range(start_year, end_year + 1):
        digitalization_level = get_digitalization_level(year)
        if digitalization_level is not None:

            is_leap_year = (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0))
            days_in_year = 366 if is_leap_year else 365
            start_date = datetime(year, 1, 1)
            for day in range(days_in_year):
                date = start_date + timedelta(days=day)
                all_data.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'digitalization_level': digitalization_level
                })


    df = pd.DataFrame(all_data)
    return df


df_daily_digitalization_level = generate_daily_digitalization_level(2017, 2021)
df_daily_digitalization_level.to_csv('digitalization_level_daily_2017_2021.csv', index=False)
print(df_daily_digitalization_level)
print("Результаты сохранены в файл 'digitalization_level_daily_2017_2021.csv'")


            date  digitalization_level
0     2017-01-01                  76.0
1     2017-01-02                  76.0
2     2017-01-03                  76.0
3     2017-01-04                  76.0
4     2017-01-05                  76.0
...          ...                   ...
1821  2021-12-27                  88.2
1822  2021-12-28                  88.2
1823  2021-12-29                  88.2
1824  2021-12-30                  88.2
1825  2021-12-31                  88.2

[1826 rows x 2 columns]
Результаты сохранены в файл 'digitalization_level_daily_2017_2021.csv'


рабочий календарь + налоговый


In [15]:
from datetime import date, timedelta
START_YEAR = 2017
END_YEAR = 2021

# Добавим новые налоги в список
TAX_TYPES = [
    "НДС", "НДФЛ", "Страховые взносы",
    "Акцизы", "Налог на имущество организаций",
    "Транспортный налог", "Налог на землю",
    "Налог на прибыль"
]

# Встроенные налоговые события (можно расширять)
TAX_EVENTS = {
    "2017-01-25": ["НДФЛ", "НДС"],
    "2017-02-15": ["Страховые взносы"],
    "2017-04-28": ["НДФЛ", "НДС"],
    "2017-05-01": ["Акцизы"],
    "2017-10-10": ["Налог на имущество организаций"],
    "2018-01-25": ["НДФЛ", "НДС"],
    "2018-02-15": ["Налог на землю"],
    "2019-04-25": ["НДС"],
    "2019-07-01": ["Транспортный налог"],
    "2020-07-15": ["НДФЛ"],
    "2021-03-15": ["НДФЛ", "Страховые взносы", "Налог на прибыль"],
}

def fetch_calendar(year):
    url = f"https://isdayoff.ru/api/getdata?year={year}&pre=1"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Ошибка при запросе к API за {year} год: {response.status_code}")
    return response.text

def get_tax_info_for_date(date_str):
    taxes_today = TAX_EVENTS.get(date_str, [])
    return {tax: int(tax in taxes_today) for tax in TAX_TYPES}

def create_calendar_dataframe():
    rows = []

    for year in range(START_YEAR, END_YEAR + 1):
        print(f"Обработка {year} года...")
        calendar_data = fetch_calendar(year)
        start_date = date(year, 1, 1)

        for i, status in enumerate(calendar_data):
            current_date = start_date + timedelta(days=i)
            current_date_str = str(current_date)

            day_type = "Рабочий" if status == "0" else "Выходной"
            is_working = 1 if status == "0" else 0

            tax_info = get_tax_info_for_date(current_date_str)
            row = {
                "date": current_date_str,
                "Тип дня": day_type,
                "Рабочий день (1/0)": is_working,
                **tax_info
            }
            rows.append(row)

    df = pd.DataFrame(rows)
    return df

calendar_df = create_calendar_dataframe()
calendar_df.to_csv("calendar_with_taxes.csv", index=False, encoding="utf-8")


Обработка 2017 года...
Обработка 2018 года...
Обработка 2019 года...
Обработка 2020 года...
Обработка 2021 года...


In [17]:
proj_file = pd.read_excel('../Project 2_2023.xlsx')
proj_file = proj_file.drop('Unnamed: 0', axis=1)
proj_file['date'] = pd.to_datetime(proj_file['date'])
proj_file['date'] = proj_file['date'].dt.strftime('%Y-%m-%d')

print(proj_file['date'].dtype)
print(proj_file.head())


object
         date    income   outcome   balance
0  2017-01-09  1.343028  1.487865 -0.144836
1  2017-01-10  1.068610  1.194182 -0.125572
2  2017-01-11  0.944429  0.936663  0.007767
3  2017-01-12  1.672202  0.875379  0.796823
4  2017-01-13  0.955924  0.975645 -0.019721


# Объединение в один DF

In [23]:
full_df

,date,CLOSE
0,2017-01-01,0.00
1,2017-01-02,0.00
2,2017-01-03,2285.43
3,2017-01-04,2263.90
4,2017-01-05,2220.35
...,...,...
1821,2021-12-27,3740.63
1822,2021-12-28,3777.56
1823,2021-12-29,3741.07
1824,2021-12-30,3787.26


In [24]:

df_combined = pd.merge(proj_file,calendar_df, on='date')
df_combined = pd.merge(df_combined,df_daily_digitalization_level, on='date')
df_combined = pd.merge(df_combined, df_daily_age_structure, on='date')
df_combined = pd.merge(df_combined, df_daily_population_density, on='date')
df_combined = pd.merge(df_combined, consumer_expenses_df , on='date')
df_combined = pd.merge(df_combined, df_daily_gdp_per_capita , on='date')
df_combined = pd.merge(df_combined, df_daily_unemployment_rate , on='date')
df_combined = pd.merge(df_combined, df_daily_trade_balance , on='date')
df_combined = pd.merge(df_combined, df_daily_inflation_rate , on='date')
df_combined = pd.merge(df_combined, full_df , on='date')
df_combined = pd.merge(df_combined, df_data , on='date')
df_combined.to_csv("final.csv", index=False, encoding="utf-8")
print(df_combined)


False
False
False
False
False
False
False
False
False
True
True
            date    income   outcome   balance  Тип дня  Рабочий день (1/0)  \
0     2017-01-09  1.343028  1.487865 -0.144836  Рабочий                   1   
1     2017-01-10  1.068610  1.194182 -0.125572  Рабочий                   1   
2     2017-01-11  0.944429  0.936663  0.007767  Рабочий                   1   
3     2017-01-12  1.672202  0.875379  0.796823  Рабочий                   1   
4     2017-01-13  0.955924  0.975645 -0.019721  Рабочий                   1   
...          ...       ...       ...       ...      ...                 ...   
1095  2021-03-25  2.304215  3.173234 -0.869018  Рабочий                   1   
1096  2021-03-26  2.172546  4.687433 -2.514887  Рабочий                   1   
1097  2021-03-29  2.356883  3.370738 -1.013855  Рабочий                   1   
1098  2021-03-30  2.317382  2.567554 -0.250172  Рабочий                   1   
1099  2021-03-31  2.528054  2.528054  0.000000  Рабочий            